In [1]:
import os
from typing import overload
import numpy as np
from pymbar import MBAR
import pandas as pd

In [2]:
os.chdir('complex_6_20_200_200_restraint_loop_no_DI-PHE_0.01/loop_restraint_on_01/post_processing_02')

In [3]:
#beta = 1.0 / (300 * 1.380649 * 6.02214076 / 1000)
beta = 1.0 / (300 * 1.380649e-23 )
name_list = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
#name_list = [ 0.0, 0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.35, 0.5, 0.75, 1.0]
#name_list = [ 0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.85, 1]

# number of states
K = len(name_list)

mian_dir = os.getcwd()

In [4]:
os.chdir(mian_dir + '/' + str(name_list[0]))
result_path = os.getcwd() + '/MBAR_energy.txt'

In [5]:
u_kn = np.loadtxt(result_path)

In [6]:
u_kn.shape

(6000,)

In [7]:
for file in range(1,len(name_list)):

    os.chdir(mian_dir + '/' + str(name_list[file]))
    result_path = os.getcwd() + '/MBAR_energy.txt'

    a = np.loadtxt(result_path)
    #print(a.shape)
    #print(a)
    u_kn = np.vstack((u_kn,a))

In [8]:
u_kn.shape

(6, 6000)

In [9]:
N_k = np.full(K,u_kn.shape[1]/K, dtype=np.int32)
N_k

array([1000, 1000, 1000, 1000, 1000, 1000], dtype=int32)

In [10]:
mbar = MBAR(u_kn,N_k)

In [11]:
Delta_f_ij, dDelta_f_ij, Theta_ij = mbar.getFreeEnergyDifferences(return_theta=True)

# kbT  -->   kcal/mol   (  *  0.596  )

In [12]:
dDelta_f_ij

array([[0.        , 0.02378343, 0.03339295, 0.03726653, 0.03958053,
        0.04142057],
       [0.02378343, 0.        , 0.01165925, 0.01736165, 0.02112974,
        0.02416069],
       [0.03339295, 0.01165925, 0.        , 0.0065609 , 0.01130458,
        0.01523458],
       [0.03726653, 0.01736165, 0.0065609 , 0.        , 0.0050607 ,
        0.00940512],
       [0.03958053, 0.02112974, 0.01130458, 0.0050607 , 0.        ,
        0.00449461],
       [0.04142057, 0.02416069, 0.01523458, 0.00940512, 0.00449461,
        0.        ]])

In [13]:
print('Delta_f_ij.shape:',Delta_f_ij.shape)
print(f"Restraint energy : {Delta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f} ± {dDelta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f} Kcal/mol")
with open(f'{mian_dir}/Restraint_energy.txt','w') as f:
    f.write(f'Restraint energy(MBAR): {Delta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f} ± {dDelta_f_ij[0,-1] / beta / 1000 / 4.186 * 6.02214076e23:.3f} Kcal/mol\n')
    for i in range(K-1):
        f.writelines(f'{Delta_f_ij[i][i+1] * 0.596:.3f}\n')

Delta_f_ij.shape: (6, 6)
Restraint energy : 3.257 ± 0.025 Kcal/mol


In [14]:
pd.DataFrame(Delta_f_ij * 0.596)

,0,1,2,3,4,5
0,0.000000,1.230946,1.959387,2.482164,2.902984,3.257548
1,-1.230946,0.000000,0.728441,1.251218,1.672038,2.026602
2,-1.959387,-0.728441,0.000000,0.522777,0.943597,1.298161
3,-2.482164,-1.251218,-0.522777,0.000000,0.420820,0.775384
4,-2.902984,-1.672038,-0.943597,-0.420820,0.000000,0.354564
5,-3.257548,-2.026602,-1.298161,-0.775384,-0.354564,0.000000


In [15]:
O_ij = mbar.computeOverlap()['matrix']
np.savetxt('../restraint_overlap.txt',O_ij,fmt='%.8f')
O_ij.round(decimals=3, out=O_ij)
pd.DataFrame(O_ij)

,0,1,2,3,4,5
0,0.545,0.235,0.102,0.056,0.036,0.026
1,0.235,0.236,0.184,0.142,0.112,0.091
2,0.102,0.184,0.200,0.188,0.171,0.156
3,0.056,0.142,0.188,0.203,0.206,0.205
4,0.036,0.112,0.171,0.206,0.229,0.245
5,0.026,0.091,0.156,0.205,0.245,0.277


In [16]:
dDelta_f_ij

array([[0.        , 0.02378343, 0.03339295, 0.03726653, 0.03958053,
        0.04142057],
       [0.02378343, 0.        , 0.01165925, 0.01736165, 0.02112974,
        0.02416069],
       [0.03339295, 0.01165925, 0.        , 0.0065609 , 0.01130458,
        0.01523458],
       [0.03726653, 0.01736165, 0.0065609 , 0.        , 0.0050607 ,
        0.00940512],
       [0.03958053, 0.02112974, 0.01130458, 0.0050607 , 0.        ,
        0.00449461],
       [0.04142057, 0.02416069, 0.01523458, 0.00940512, 0.00449461,
        0.        ]])